# Denoise Lab

In [1]:
import skimage
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import gridspec


from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.restoration import (calibrate_denoiser,
                                 denoise_wavelet,
                                 denoise_tv_chambolle, denoise_nl_means,
                                 estimate_sigma)
from bcd.dal.io.image import ImageIO

In [3]:
denoise_nl_means.__name__

'denoise_nl_means'

In [2]:
image = ImageIO.read(filepath = "data/image/1_dev/4a104faa-1b67-4c4a-9769-6a066ea7cb8a.png")

## Total Variation

In [3]:
parameter_ranges_tv = {'weight': np.arange(0.05, 0.3, 0.2)}
_, (parameters_tested_tv, losses_tv) = calibrate_denoiser(
                                    image,
                                    denoise_tv_chambolle,
                                    denoise_parameters=parameter_ranges_tv,
                                    extra_output=True)
print(f'Minimum self-supervised loss TV: {np.min(losses_tv):.4f}')                      
best_parameters_tv = parameters_tested_tv[np.argmin(losses_tv)]
denoised_calibrated_tv = denoise_invariant(image, denoise_tv_chambolle,
                                           denoiser_kwargs=best_parameters_tv)
denoised_default_tv = denoise_tv_chambolle(image, **best_parameters_tv)

psnr_calibrated_tv = psnr(image, denoised_calibrated_tv)
psnr_default_tv = psnr(image, denoised_default_tv)              

## Wavelet

In [ ]:
parameter_ranges_wavelet = {'sigma': np.arange(0.01, 0.3, 0.03)}
_, (parameters_tested_wavelet, losses_wavelet) = calibrate_denoiser(
                                                image,
                                                _denoise_wavelet,
                                                parameter_ranges_wavelet,
                                                extra_output=True)
print(f'Minimum self-supervised loss wavelet: {np.min(losses_wavelet):.4f}')

best_parameters_wavelet = parameters_tested_wavelet[np.argmin(losses_wavelet)]
denoised_calibrated_wavelet = denoise_invariant(
        image, _denoise_wavelet,
        denoiser_kwargs=best_parameters_wavelet)
denoised_default_wavelet = _denoise_wavelet(image, **best_parameters_wavelet)

psnr_calibrated_wavelet = psnr(image, denoised_calibrated_wavelet)
psnr_default_wavelet = psnr(image, denoised_default_wavelet)

## Non-Local Means

In [ ]:
sigma_est = estimate_sigma(image)

parameter_ranges_nl = {'sigma': np.arange(0.6, 1.4, 0.2) * sigma_est,
                       'h': np.arange(0.6, 1.2, 0.2) * sigma_est}

parameter_ranges_nl = {'sigma': np.arange(0.01, 0.3, 0.03)}
_, (parameters_tested_nl, losses_nl) = calibrate_denoiser(image,
                                                        denoise_nl_means,
                                                        parameter_ranges_nl,
                                                        extra_output=True)
print(f'Minimum self-supervised loss NL means: {np.min(losses_nl):.4f}')

best_parameters_nl = parameters_tested_nl[np.argmin(losses_nl)]
denoised_calibrated_nl = denoise_invariant(image, denoise_nl_means,
                                           denoiser_kwargs=best_parameters_nl)
denoised_default_nl = denoise_nl_means(image, **best_parameters_nl)

psnr_calibrated_nl = psnr(image, denoised_calibrated_nl)
psnr_default_nl = psnr(image, denoised_default_nl)

## Bilateral 
### Parameters
Performance of the bilateral filter is governed by two parameters $\theta_d$ and $\theta_r$. The $\theta_d$ parameter determines the kernel size and a rule of thumb, is to set this to 2% of the image diagonal. The $\theta_r$ is the standard deviation of the range and should be proportional to the noise in the image. A rule of thumb is to set $\theta_r$ to the mean or median of the image gradients.

# Results

In [ ]:
print('                       PSNR')
print(f'NL means (Default)   : {psnr_default_nl:.1f}')
print(f'NL means (Calibrated): {psnr_calibrated_nl:.1f}')
print(f'Wavelet  (Default)   : {psnr_default_wavelet:.1f}')
print(f'Wavelet  (Calibrated): {psnr_calibrated_wavelet:.1f}')
print(f'TV norm  (Default)   : {psnr_default_tv:.1f}')
print(f'TV norm  (Calibrated): {psnr_calibrated_tv:.1f}')

plt.subplots(figsize=(10, 12))
plt.imshow(image, cmap='Greys_r')
plt.xticks([])
plt.yticks([])
plt.title('Noisy Image')

def get_inset(x):
    return x[0:100, -140:]

fig, axes = plt.subplots(ncols=3, nrows=2, figsize=(15, 8))

for ax in axes.ravel():
    ax.set_xticks([])
    ax.set_yticks([])

axes[0, 0].imshow(get_inset(denoised_default_nl), cmap='Greys_r')
axes[0, 0].set_title('NL Means Default')
axes[1, 0].imshow(get_inset(denoised_calibrated_nl), cmap='Greys_r')
axes[1, 0].set_title('NL Means Calibrated')
axes[0, 1].imshow(get_inset(denoised_default_wavelet), cmap='Greys_r')
axes[0, 1].set_title('Wavelet Default')
axes[1, 1].imshow(get_inset(denoised_calibrated_wavelet), cmap='Greys_r')
axes[1, 1].set_title('Wavelet Calibrated')
axes[0, 2].imshow(get_inset(denoised_default_tv), cmap='Greys_r')
axes[0, 2].set_title('TV Norm Default')
axes[1, 2].imshow(get_inset(denoised_calibrated_tv), cmap='Greys_r')
axes[1, 2].set_title('TV Norm Calibrated')

for spine in axes[1, 2].spines.values():
    spine.set_edgecolor('red')
    spine.set_linewidth(5)

plt.show()